# Study geometry handling
Check if the tools used to handle the geometry are good enough

## TODO
- ?

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import timeit
import copy
import concurrent.futures 
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import batoid

from ghosts import plotter
from ghosts import  simulator
from ghosts import tweak_optics

from ghosts.analysis import compute_ghost_separations, reduce_ghosts
from ghosts.analysis import match_ghosts, compute_reduced_distance, compute_2d_reduced_distance, find_nearest_ghost
from ghosts.analysis import make_data_frame

from ghosts.beam_configs import BEAM_CONFIG_0, BEAM_CONFIG_1, BEAM_CONFIG_2, FAST_BEAM_CONFIG_1

from ghosts import geom
from ghosts.geom_config import GEOM_CONFIG_0

In [ ]:
from ghosts.constants import *
print(f'LSST Camera has {LSST_CAMERA_PIXEL_DENSITY_MM2:.0f} pixels per mm^2')

In [ ]:
# build the standard telescope from GEOM_CONFIG_0
telescope = tweak_optics.build_telescope_from_geom(GEOM_CONFIG_0)

In [ ]:
# Run simulation with standard beam config
traceFull, rForward, rReverse, rays = simulator.run_simulation(telescope, beam_config=BEAM_CONFIG_1)
simulation = [traceFull, rForward, rReverse, rays]
# Check setup
plotter.plot_setup(telescope, simulation)


In [ ]:
# Zoom on ghosts
plotter.plot_zoom_on_ghosts(rForward)

In [ ]:
# Analyze reference set of beam spots
ref_spots_data, _ref_spots = reduce_ghosts(rForward)
ref_data_frame = make_data_frame(ref_spots_data, BEAM_CONFIG_1)
ref_data_frame.sort_values(by=['name'])

Now the exercise consists in building several geometries, store these in a panda data frame, and then produce simulations from the data frame

In [ ]:
def run_and_analyze_simulation_for_geom_frame(geom_data_frame, beam_config=BEAM_CONFIG_1):
    # define outputs
    beam_spots = list()
    # list of unique geom ids
    geom_id_set = set(geom_data_frame['geom_id'])
    # loop on geom ids
    for geom_id in geom_id_set:
        # get geom config as a dictionary for the geom id
        geom_config = geom.to_dict(geom_data_frame[geom_data_frame['geom_id']==geom_id])
        # build the standard telescope from GEOM_CONFIG_0
        telescope = tweak_optics.build_telescope_from_geom(geom_config)
        # run simulation
        print(f'Run simulation for geom id: {geom_id}')
        trace_full, r_forward, r_reverse, rays = simulator.run_simulation(telescope, beam_config=beam_config)
        simulation = [trace_full, r_forward, r_reverse, rays]
        # analyze beam spots
        spots_data, _spots = reduce_ghosts(r_forward)
        spots_data_frame = make_data_frame(spots_data, beam_config)
        data = (geom_id, spots_data_frame)
        beam_spots.append(data)
    # return the set of beam spots and the geom id
    return beam_spots

In [ ]:
my_shifts = [dx*0.01 for dx in range(-10, 11)]
g = geom.build_translation_set('L1', 'x', my_shifts, 10000)

In [ ]:
frame = geom.concat_dicts(g)
frame.tail()


In [ ]:
one = frame[frame['geom_id']==10015]
one

In [ ]:
geom.to_dict(one)

In [ ]:
geom_id_set = set(frame['geom_id'])

In [ ]:
# run simulations for the set of geometries
beam_spots = run_and_analyze_simulation_for_geom_frame(frame, beam_config=BEAM_CONFIG_1)

In [ ]:
# Compute distances for each frame
distances_2d = list()
distances_3d = list()
my_geom_ids = list()
for one in beam_spots:
    geom_id = one[0]
    df_i = one[1]
    my_geom_ids.append(geom_id)
    match_ref = match_ghosts(ref_data_frame, df_i)
    distances_2d.append(compute_2d_reduced_distance(match_ref))
    distances_3d.append(compute_reduced_distance(match_ref))


In [ ]:
plotter.plot_distances_for_scan(my_geom_ids, distances_2d, distances_3d)

In [ ]:
# Test rotation set
my_angles = [i*0.01 for i in range(-10, 11)]
r_set = geom.build_rotation_set('L2', 'y', my_angles, 20000)
r_set_frame = geom.concat_dicts(r_set)
r_set_frame.tail()

In [ ]:
# run simulations for the set of geometries
r_beam_spots = run_and_analyze_simulation_for_geom_frame(r_set_frame, beam_config=BEAM_CONFIG_1)

In [ ]:
# Compute distances for each frame
r_distances_2d = list()
r_distances_3d = list()
r_my_geom_ids = list()
for one in r_beam_spots:
    geom_id = one[0]
    df_i = one[1]
    r_my_geom_ids.append(geom_id)
    match_ref = match_ghosts(ref_data_frame, df_i)
    r_distances_2d.append(compute_2d_reduced_distance(match_ref))
    r_distances_3d.append(compute_reduced_distance(match_ref))


In [ ]:
plotter.plot_distances_for_scan(r_my_geom_ids, r_distances_2d, r_distances_3d)